<a href="https://colab.research.google.com/github/suaJeong-777/AI-recommend-system/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/소개딩_2022/tmdb_product.csv', encoding='cp949',index_col=0)
df

,title
id,
19995,플리츠 백팩
285,심플파우치
206647,프린트 클러치
49026,백팩
49529,오브티
559,수영복
38757,명함지갑
99861,대나무칫솔
767,설거지비누


In [10]:
import sklearn
import pandas as pd
import warnings; warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

movies = pd.read_csv('/content/drive/MyDrive/소개딩_2022/tmdb_5000_environment.csv',encoding='cp949')

movies_df = movies[['id','title','genres','voteAverage', 'voteCount', 'popularity', 'keywords', 'overview']]

from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])

from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word단위가 구분되는 문자열로 변환.
movies_df['generes_literal'] = movies_df['genres'].apply(lambda x :  (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['generes_literal'])

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)

genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]

def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    #인자로 입력된 movies_df DataFrame에서 'title'칼럼이 입력된 title_name값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]

    #title_named을 가진 DataFrame의 index객체를 ndarray로 반환하고
    #sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    #추출된 top_n index출력, top_n index는 2차원 데이터임
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

def movie_list(products):
  similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, products, 5)  #찾고자 하는 데이터와 유사한 것
  return (similar_movies[['title']]) 

test_results= "티백"  #추후 테스트 결과값을 받아오는 부분(수정 필요)



In [30]:
import sklearn
import pandas as pd
import warnings; warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

movies = pd.read_csv('/content/drive/MyDrive/소개딩_2022/tmdb_5000_environment.csv',encoding='cp949')

movies_df = movies[['id','title','genres','voteAverage', 'voteCount', 'popularity', 'keywords', 'overview']]

from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])

from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word단위가 구분되는 문자열로 변환.
movies_df['generes_literal'] = movies_df['genres'].apply(lambda x :  (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['generes_literal'])

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)

genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]

def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    #인자로 입력된 movies_df DataFrame에서 'title'칼럼이 입력된 title_name값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]

    #title_named을 가진 DataFrame의 index객체를 ndarray로 반환하고
    #sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    #추출된 top_n index출력, top_n index는 2차원 데이터임
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

def movie_list(products):
  similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, products, 5)  #찾고자 하는 데이터와 유사한 것
  return (similar_movies[['title']]) 

test_results= "PCR 펜홀더"  #추후 테스트 결과값을 받아오는 부분(수정 필요)



In [34]:
print(movie_list(test_results))

[[72 73 74 75 48]]
               title
72  재생펠트 펜트레이 멜란지그레이
73   재생펠트 펜트레이 다크그레이
74     재생펠트 펜트레이 오렌지
75           PCR 펜홀더
48              비누받침


In [35]:
list_products= movie_list(test_results).to_dict()
recommend_product=list_products['title'].values()
recommend_list=list(recommend_product)

[[72 73 74 75 48]]


In [36]:
movie_list(test_results)['title'].to_list()

[[72 73 74 75 48]]


['재생펠트 펜트레이 멜란지그레이', '재생펠트 펜트레이 다크그레이', '재생펠트 펜트레이 오렌지', 'PCR 펜홀더', '비누받침']

In [37]:
print(movie_list(test_results).iloc[4].to_list())

[[72 73 74 75 48]]
['비누받침']


In [56]:
test= str(movie_list(test_results).iloc[4].to_list())
test

[[72 73 74 75 48]]


"['비누받침']"

In [51]:
test.replace('[','')
test

"['비누받침']"

In [57]:
len(test)

8

In [59]:
test[2:-2]

'비누받침'

In [53]:
test2= str(movie_list(test_results).iloc[3].to_list())
test2

[[72 73 74 75 48]]


"['PCR 펜홀더']"

In [54]:
len(test2)

11

In [60]:
test2[2:-2]

'PCR 펜홀더'

In [19]:
type(movie_list(test_results))

[[23 45 44 43 17]]


pandas.core.frame.DataFrame

In [20]:
list_product= movie_list(test_results).values.tolist()

[[23 45 44 43 17]]


In [21]:
type(list_product)

list

In [13]:
movie_list(test_results).title

[[23 45 44 43 17]]


23       티백
45     작두콩차
44      우엉차
43      비트차
17    삼베마스크
Name: title, dtype: object